In [4]:
import requests
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.express as px
from io import StringIO
import plotly.graph_objs as go

In [5]:
def fetch_data(endpoint):
    url = f"http://apps.who.int/gho/athena/api/GHO/{endpoint}?format=csv"
    response = requests.get(url)
    print(f'Fetched data for {endpoint}')
    csv_data = StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(csv_data)
    df = keep_most_recent_date(df).loc[:, ['COUNTRY', 'YEAR', 'Numeric','REGION']]
    return df

def keep_most_recent_date(df):
    df = df.sort_values(by=['COUNTRY', 'YEAR'], ascending=[True, False])
    df = df.drop_duplicates(subset='COUNTRY', keep='first')
    return df

In [6]:
stunting = "NUTRITION_ANT_HAZ_NE2"

In [7]:
df = fetch_data(stunting)


Fetched data for NUTRITION_ANT_HAZ_NE2


In [52]:
df.groupby('COUNTRY')['Numeric'].mean().reset_index().sort_values(by='Numeric', ascending=False).iloc[:5,:]

,COUNTRY,Numeric
42,ERI,52.5
7,BDI,50.9
107,PNG,49.5
136,TLS,46.7
54,GTM,46.7


In [44]:
grouped_df = df.groupby('COUNTRY')['Numeric'].mean().reset_index()
max_index = grouped_df['Numeric'].idxmax()
most_affected_country = grouped_df.loc[max_index,'COUNTRY']
df['YEAR'] = df['YEAR'].astype(int)
df[df['COUNTRY'] == most_affected_country].groupby('YEAR')['Numeric'].mean().reset_index(drop=False).sort_values(by="YEAR")

,YEAR,Numeric
0,2010,52.5
